# Tekstanalyse 2: Named entities regognition (NER) og Parts of Speech Tagging (POS) med SpaCy
***
***
Keywords: `context manager`, `named entity recognition`, `NER`, `parts of speech tagging`, `POS`, `WordCloud`

Nye Python-udtryk:  `with`, `.set()`, `.sorted()`, `.join()`, `WordCloud()`
***
***
I det følgende skal vi se nærmere på NLP-pakken `SpaCy`, der er indeholder effektive og kraftfulde redskaber til tekstanalyse. I modsætning til `NLTK`-pakken, der er regelbaseret, er SpaCy baseret på maskinlæring, hvilket bl.a. betyder, at den virker godt på dansk, for sprogmodulet også er trænet på danske tekster. 

SpaCy kan mange forskellige ting. Vi lægger ud med at kigge på **Named Entity Recognition** (NER) og **Parts of Speech Tagging**.

Nogle af elementerne vil være repetition af elementer fra tidligere notebooks.

Hvis der er kode sekvenser eller udtryk i ikke forstår, er det altid en god idé at bruge Google. Det kan give svar på det meste.

### Husk at kommentere eksemplerne, så I også kan læse og forstå dem senere

# 1. Forberedelse

### Dependencies
Som altid begynder vi med at importere de nødvendige `libraries`. Udover `os`, `Numpy` og `Pandas` skal vi også bruge `matplotlib` og `nltk`.

In [1]:
import os                       # os tillader os bl.a. at finde filplaceringer på computeren
import numpy as np              # Numpy leverer noget af matematikken, der ligger under Pandas 
import pandas as pd             # Pandas tillader os at importere, oprette og manipulere data frames
import matplotlib.pyplot as plt # Importerer underbiblioteket pyplot fra pakken matplotlib
from nltk.text import Text      # nltk indholder mange forskellige funktioner, der kan bruges til tekstanalyse

Herudover skal vi også importere `SpaCy`. Når vi bruger spacy, skal vi udover at importere modulet, også loade en sprogmodel. Der er tre modeller at vælge mellem: `da_core_news_sm`, `da_core_news_md`, `da_core_news_lg`, en lille (sm), en mellem (md) og en stor (lg). Størrelsen angiver, hvor stort et korpus modellen er blevet trænet på. 

Hvis I ikke allerede har gjort det skal, skal I, første gang I bruger SpaCy, downloade de modeller, I vil bruge. I kan enten gøre det fra kommandoprompten eller direkte fra jeres Notebook. Hvis i gør det fra Notebok'en skal I sætte `!` foran kommandoen (se nedenfor). Det angiver at der er en command line-kommando. Hvis I åbner kopmmandoprompten, skal I bruge samme kommando, men undlade `!`.

In [12]:
!python -m spacy download da_core_news_sm

[+] Download and installation successful
You can now load the package via spacy.load('da_core_news_sm')


In [13]:
!python -m spacy download da_core_news_md

[+] Download and installation successful
You can now load the package via spacy.load('da_core_news_md')


In [14]:
!python -m spacy download da_core_news_lg

[+] Download and installation successful
You can now load the package via spacy.load('da_core_news_lg')


**Importér** SpaCy og **load** den store danske sprog-model. Den kører derfor lidt langsommere end de andre, men er til gengæld mere præcis. Skal man arbejde med meget store tekstmængder, kan det være en idé at bruge en mindre model. Det er en afvejning af regnekraft/tid mod præcision.

In [15]:
import spacy          

In [16]:
nlp = spacy.load("da_core_news_lg")

### Import af tekster
I denne notebook skal vi arbejde med seks forskellige nytårstaler. To af Mette Frederiksen, to af Lars Løkke Rasmussen og to af Helle Thorning Schmidt. **Placér** de downloadede `.txt`-filer i en mappe i den mappe, hvori i har gemt jeres script.

#### Context manager
Der er forskellige måder at åbne filer på. Nedenfor finder I et eksempel på en såkaldt `context manager`. En af fordelene ved at bruge en context-manager er, at den automatisk lukker filerne igen efter de er blevet indlæst. 

Kodesekvensen begynder med et `for loop`, fordi vi skal åbne seks forskellige filer. Herefter følger context-manageren, der læses 'brug kommandoen 'open' til at åbne den angivne fil og gem den under variabelnavnet 'f' (f for fil er standard-notation, men det er en variabel, så navnet er valgfrit)'.

Herefter tilføjer vi talerne til listen 'Taler'. **Bemærk** at rensning foregår i samme ombæring med `.replace()`. Teksten i .txt-filerne er allerede nogenlunde rensede, og det er derfor ikke nødvendigt at bruge rense-funktionerne. Denne måde virker bedst, hvis det kun er få ting, der skal gøres. Er rensningen mere omfattende, er det mere overskueligt, at definere en pipeline-funktion.

In [17]:
taler = [] # opretter tom liste

path = os.path.join("Taler") # bruges for at undgå mac/pc-problemerne med absolutte stinavne

for fil in os.scandir(path): # for-loop
    with open (fil, encoding = "utf8") as f: # context manager
        taler.append(f.read().replace("\n"," ").replace("*"," ")) # tilføj renset tekst til liste

For overskuelighedens skyld, lægger vi ud med at afprøve SpaCy på en enkelt tale. Vi gemmer talen under variabel-navnet 'tale_1'.

**Bemærk** notationen `taler[0]`. **Hvad** betyder det, og **hvordan** læses det?

In [18]:
tale_1 = taler[0]

# SpaCy

Vi har allerede importeret `SpaCy` og load'et den store danske sprogmodel. Modellen blev gemt under navnet `nlp`. Det er standardnotation, men betegnelsen er valgfri, og I kan ændre det, hvis det er nødvendigt, fx hvis I har load'et flere forskellige modeller.

Bag navnet 'nlp' gemmer der sig nu en masse funktionalitet, på samme måde som vi tidligere har pakket funktionalitet i vores pipeline-funktioner. Når I anvender 'nlp' på en tekst, gør den derfor mange ting på en gang, og outputtet, som vi gemmer under navnet 'doc1' er et komplekst objekt, der udover teksten indholder en masse tags og meta-data.

In [19]:
doc1 = nlp(tale_1)

Vi har ikke umiddelbart adgang til metainformationen. Prøv at taste `doc1` og `print(doc1)` i feltet nedenfor, og **diskutér** hvad I ser.

In [23]:
doc1

HTS_2012  Godaften.  For tre måneder siden fik Danmark en ny regering. En regering, som er trådt til i en krisetid. Og som vil tage fat på de alvorlige problemer, Danmark står med.  Vores mål er at bringe Danmark sikkert gennem krisen og ud på den anden side med vores velstand, ordentlighed og omsorg for hinanden i behold.  Det bliver ikke nemt. Men vi kan hvis vi vil.  Det nye år – 2012 – bliver ikke året, hvor vi kommer fri af problemerne. Det må vi allerede se i øjnene nu.  Men med fælles hjælp kan det blive året, hvor vi med seje skridt vender udviklingen.  Generation efter generation har vi skabt fremskridt for almindelige mennesker. Og vi har insisteret på, at alle skulle med. På den måde er Danmark blevet et fantastisk land.  Min mormor blev født i år 1900. Da hun var 15 år, fik kvinder og tjenestefolk stemmeret. Demokratiet blev bredt ud til almindelige danskere.  Min mor blev født i 1937. Mens hun voksede op, kom Danmark fri af krigen og rationeringsmærkerne, og efterhånden fi

# 2. Named Entity Regonition

Det første vi skal se på er `named entity recognition`, som er en funktion, der gør det muligt at genkende personer, organisationer, steder mm.

For at hente informationen ud af vore `doc`-objekt, skal vi bruge `.ents`-kommandoen, der er nemmest at arbejde med, hvis vi gemmer informationen på en liste.

**Afprøv** sekvensen nedenfor. **Læs** koden og **diskutér**, hvad de enkelte dele betyder.

In [24]:
ents1 = list(doc1.ents)

Hvis vi bare printer indholdet af variablen 'ents1', får vi blot en ordliste. Vi er derfor nødt til specifikt at bede om de enkelte elementer vha. `.text` og `.label_`-kopmmandoerne.

**Afprøv** sekvensen nedenfor. **Læs** koden og **diskutér**, hvad de enkelte dele betyder.

In [25]:
for w in ents1:
    print(w.text,w.label_)

HTS_2012 ORG
Danmark LOC
Danmark LOC
Danmark LOC
Danmark LOC
Demokratiet MISC
danskere MISC
Danmark LOC
Danmark LOC
danskerne MISC
Danmark LOC
Danmark LOC
Danmark LOC
Danmark LOC
Danske MISC
Danmark LOC
Danmark LOC
danske MISC
Europa LOC
europæerne MISC
danske MISC
Danmark LOC
Danmark LOC
danskere MISC
Danmark LOC
danskere MISC
Danmark LOC
dansk MISC
Aalborg LOC
Søren PER
Søren PER
Midt- LOC
Nordjylland LOC
Danmark LOC
danskere MISC
danske MISC
Afghanistan LOC
Danmark LOC
Afghanistan LOC
Danmarks LOC
Afghanistan LOC
Libyen LOC
Danmark LOC
arabiske MISC
Danmark LOC
Danmark LOC
Danmark LOC
EU ORG
Danmark LOC
EU ORG
europæiske MISC
Europa LOC
Europa LOC
Europa LOC
Danmarks LOC
danskernes MISC
Europa LOC
Danmark LOC
Danmark LOC


Hvis I er usikre på, hvad et SpaCy-tag betyder kan I altid bruge `spacy.explain()`, fx:

In [26]:
spacy.explain("LOC")

'Non-GPE locations, mountain ranges, bodies of water'

## Personer
På listen ovenfor har vi en komplet oversigt over de ord, som SpaCy har vurderet betegner personer, steder osv. Når I skal bruge det analytiske, skal i naturligvis tjekke, om der er fejl. SpaCy performer godt, men der er pt. ingen modeller, der kan levere 100% præcision.

Hvis vi vil udtrække en liste over personer, kan vi bruge `.label_` til at sortere med.

I eksemplet nedenfor bruger vi `list comprehension`til at lave en liste, samt `.set()` til at slette dubletter, og `.sorted()` til at sortere outputtet. Et `set` (en mængde) er karakteriseret ved, at den kun indholde ét eksemplar af hvert element. Når vi transformerer en liste til en mængde, sletter vi derfor automatisk alle ord-dubletter. Kommandoen `.sorted()`transformerer herefter mængden til en sorteret liste.

Som vi så sidst fungerer `list comprehension` lidt som et `for loop`. Vi laver altså en liste af de tekstelementer (.text)fra listen 'ents1', der har et label (.label_), der er lig med "PER". Herefter slettes dubletter, og listen sorteres alfabetisk.

**Afprøv** kode sekvensen nedenfor. **Læs** koden og **diskutér** hvad de enkelte betyder.

In [27]:
personer = sorted(set([t.text for t in ents1 if t.label_ == "PER"]))

**Print** listen for at se resultatet.

In [28]:
print(personer)

['Søren']


# Opgave 1
Lav tilsvarende lister, hvor i sorterer efter `"LOC"` (steder) og `"MISC"` (miscellaneous/diverse).

## Steder

## Diverse (miscellaneous)

# Opgave 2
Vælg en tale af henholdsvis Lars Løkke Rasmussen og Mette Frederiksen og lav for hver af de to taler en NER-analyse samt lister med personer, steder og diverse.

**Husk** at I allerede har indlæst alle taler under variabelnavnet `taler`

**Husk også** at give variablerne nye navne, fx `personer_LLR` og `personer_MF`, så I ikke overskriver de lister, I allerede har lavet!

#### Lars Løkke rasmussen

#### Mette Frederiksen

# Opgave 3
**Sammenlign** listerne for de tre taler. **Diskutér** forskelle og ligheder i indholdet af talerne. **Hvordan** kan vi bruge denne information til at karakterisere teksterne.


# 3. Parts of Speech Tagging (POS)

Det næste vi skal se på er SpaCy's POS-modul. `Parts of Speech Tagging` er en funktion, der gør det muligt at opmærke alle ord med ordklasse-tags. 

For at hente informationen ud af vore doc-objekt, skal vi bruge `.sents`-kommandoen. Der er ligesom sidst nemmest at håndtere, hvis vi gemmer informationen på en liste.

**Afprøv** sekvensen nedenfor. **Læs** koden og **diskutér** hvad de enkelte dele betyder.


In [29]:
sents1 = list(doc1.sents)[1:]

**Hvad** er konsekvensen af at tilføje `[1:]`. **Lav** eventuelt en liste, hvor I udlader dette. Sammenlign de to lister. **Hvad** er forskellen?

In [30]:
sents0 = list(doc1.sents)

In [33]:
print(sents0)

[HTS_2012  Godaften.,  For tre måneder siden fik Danmark en ny regering., En regering, som er trådt til i en krisetid., Og som vil tage fat på de alvorlige problemer, Danmark står med.,  Vores mål er at bringe Danmark sikkert gennem krisen og ud på den anden side med vores velstand, ordentlighed og omsorg for hinanden i behold.,  , Det bliver ikke nemt., Men vi kan hvis vi vil.,  Det nye år – 2012 – bliver ikke året, hvor vi kommer fri af problemerne., Det må vi allerede se i øjnene nu.,  Men med fælles hjælp kan det blive året, hvor vi med seje skridt vender udviklingen.,  Generation efter generation har vi skabt fremskridt for almindelige mennesker., Og vi har insisteret på, at alle skulle med., På den måde er Danmark blevet et fantastisk land.,  Min mormor blev født i år 1900., Da hun var 15 år, fik kvinder og tjenestefolk stemmeret., Demokratiet blev bredt ud til almindelige danskere.,  Min mor blev født i 1937., Mens hun voksede op, kom Danmark fri af krigen og rationeringsmærkern

In [34]:
print(sents1)

[ For tre måneder siden fik Danmark en ny regering., En regering, som er trådt til i en krisetid., Og som vil tage fat på de alvorlige problemer, Danmark står med.,  Vores mål er at bringe Danmark sikkert gennem krisen og ud på den anden side med vores velstand, ordentlighed og omsorg for hinanden i behold.,  , Det bliver ikke nemt., Men vi kan hvis vi vil.,  Det nye år – 2012 – bliver ikke året, hvor vi kommer fri af problemerne., Det må vi allerede se i øjnene nu.,  Men med fælles hjælp kan det blive året, hvor vi med seje skridt vender udviklingen.,  Generation efter generation har vi skabt fremskridt for almindelige mennesker., Og vi har insisteret på, at alle skulle med., På den måde er Danmark blevet et fantastisk land.,  Min mormor blev født i år 1900., Da hun var 15 år, fik kvinder og tjenestefolk stemmeret., Demokratiet blev bredt ud til almindelige danskere.,  Min mor blev født i 1937., Mens hun voksede op, kom Danmark fri af krigen og rationeringsmærkerne, og efterhånden fik

Som vi gjorde ovenfor, kan vi hente tekst og tag ud af listen vha. `.text`og `.pos_`.

Kodesekvensen nedenfor giver os en komplet liste (og den er lang).

**Afprøv** sekvensen nedenfor. **Læs** koden og **diskutér**, hvad de enkelte dele betyder.

In [35]:
for s in sents1:
    for w in s:
        print(w.text, w.pos_)

  SPACE
For ADP
tre NUM
måneder NOUN
siden ADV
fik VERB
Danmark PROPN
en DET
ny ADJ
regering NOUN
. PUNCT
En DET
regering NOUN
, PUNCT
som PRON
er AUX
trådt VERB
til ADV
i ADP
en DET
krisetid NOUN
. PUNCT
Og CCONJ
som ADP
vil AUX
tage VERB
fat ADV
på ADP
de DET
alvorlige ADJ
problemer NOUN
, PUNCT
Danmark PROPN
står VERB
med ADP
. PUNCT
  SPACE
Vores DET
mål NOUN
er VERB
at PART
bringe VERB
Danmark PROPN
sikkert ADV
gennem ADP
krisen NOUN
og CCONJ
ud ADV
på ADP
den DET
anden DET
side NOUN
med ADP
vores DET
velstand NOUN
, PUNCT
ordentlighed NOUN
og CCONJ
omsorg NOUN
for ADP
hinanden PRON
i ADP
behold NOUN
. PUNCT
  SPACE
Det PRON
bliver AUX
ikke ADV
nemt ADJ
. PUNCT
Men CCONJ
vi PRON
kan AUX
hvis SCONJ
vi PRON
vil VERB
. PUNCT
  SPACE
Det DET
nye ADJ
år NOUN
– AUX
2012 NUM
– PUNCT
bliver AUX
ikke ADV
året NOUN
, PUNCT
hvor ADV
vi PRON
kommer VERB
fri ADJ
af ADP
problemerne NOUN
. PUNCT
Det PRON
må AUX
vi PRON
allerede ADV
se VERB
i ADP
øjnene NOUN
nu ADV
. PUNCT
  SPACE
Men CCONJ
med A

vores DET
skattesystem NOUN
- PUNCT
finansieret VERB
krone NOUN
for ADP
krone NOUN
. PUNCT
Alt ADJ
sammen ADV
for ADP
at PART
få VERB
flere ADJ
i ADP
arbejde NOUN
og CCONJ
gøre VERB
vores DET
økonomi NOUN
mere ADV
robust ADJ
. PUNCT
  SPACE
Mange ADJ
vil AUX
opleve VERB
forandringer NOUN
. PUNCT
Mange ADJ
vil AUX
blive AUX
bedt VERB
at PART
tage VERB
en DET
tørn NOUN
ekstra ADJ
. PUNCT
  SPACE
Og CCONJ
ja INTJ
, PUNCT
vi PRON
vil AUX
opleve VERB
nedskæringer NOUN
og CCONJ
besparelser NOUN
. PUNCT
  SPACE
I ADP
den DET
kommende VERB
tid NOUN
skal AUX
vi PRON
tage VERB
beslutninger NOUN
, PUNCT
der PRON
kan AUX
måle VERB
sig PRON
med ADP
de DET
sværeste ADJ
i ADP
vores DET
historie NOUN
. PUNCT
  SPACE
Men CCONJ
vi PRON
gør VERB
det PRON
, PUNCT
fordi SCONJ
det PRON
er AUX
nødvendigt ADJ
. PUNCT
Vi PRON
gør VERB
det PRON
ikke ADV
, PUNCT
fordi SCONJ
vi PRON
er AUX
holdt VERB
op ADV
med ADP
at PART
tro VERB
på ADP
solidaritet NOUN
. PUNCT
Men CCONJ
netop ADV
fordi SCONJ
vi PRON
stadig ADV

er AUX
anerkendt VERB
for ADP
vores DET
brede ADJ
indsats NOUN
og CCONJ
for ADP
at PART
påtage VERB
os PRON
et DET
ansvar NOUN
, PUNCT
der PRON
rækker VERB
langt ADV
ud ADV
over ADP
vores DET
grænser NOUN
og CCONJ
langt ADV
ud ADV
over ADP
vores DET
størrelse NOUN
. PUNCT
Og CCONJ
det PRON
giver VERB
respekt NOUN
ude ADV
i ADP
verden NOUN
. PUNCT
Danmark PROPN
nyder VERB
høj ADJ
anseelse NOUN
. PUNCT
Og CCONJ
den DET
anseelse NOUN
har AUX
vi PRON
alle ADJ
lod NOUN
og CCONJ
del NOUN
i ADP
. PUNCT
  SPACE
Derfor ADV
er VERB
der ADV
også ADV
store ADJ
forventninger NOUN
til ADP
Danmark PROPN
, PUNCT
når SCONJ
vi PRON
i ADP
dag NOUN
den DET
1. ADJ
januar NOUN
overtager VERB
formandskabet NOUN
for ADP
EU PROPN
. PUNCT
  SPACE
Danmark PROPN
skal AUX
lede VERB
EU PROPN
på ADP
et DET
meget ADV
vanskeligt ADJ
tidspunkt NOUN
. PUNCT
Der ADV
ligger VERB
store ADJ
opgaver NOUN
foran ADP
os PRON
. PUNCT
Og CCONJ
dem PRON
vil AUX
vi PRON
arbejde VERB
hårdt ADV
på ADP
at PART
løfte VERB
. PUNCT
Vi PR

## Substantiver
Vi kan lave en en liste over substantiver ved at sortere efter POS-tagget "NOUN".

Kodesekvensen nedenfor indeholde ene kendte elementer. 


In [36]:
subst_1 = []
for s in sents1: # loop 1
    for w in s:  # loop 2
        if w.pos_ == "NOUN": # condition
                subst_1.append(w.text) # output

print(len(subst_1))

print(sorted(set(subst_1)))

388
['Balancen', 'Demokratiet', 'Firmaets', 'Forbruget', 'Generation', 'Gæster', 'Krisen', 'Meget', 'Midt-', 'Regningen', 'Skiftet', 'Telefoner', 'Underskuddet', 'Utryghed', 'Virksomheder', 'afslag', 'aften', 'alvor', 'anseelse', 'anstrengelse', 'ansvar', 'ansvarsfulde', 'ansøgning', 'appelsiner', 'arbejde', 'arbejdsgivere', 'arbejdsmarkedet', 'arbejdspladser', 'baggrund', 'balance', 'bank', 'befolkningen', 'begivenheder', 'behandling', 'behold', 'bekymring', 'beslutninger', 'besparelser', 'boligpriserne', 'brug', 'bud', 'budgetter', 'budgettet', 'børn', 'børnehaver', 'chauffør', 'chefgangen', 'dag', 'danskere', 'danskerne', 'danskernes', 'december', 'del', 'dele', 'demokratiets', 'europæerne', 'evne', 'familie', 'familiers', 'familievirksomheder', 'fattigdom', 'finanser', 'finansfolk', 'flæskesteg', 'folk', 'forandringer', 'formandskabet', 'forpligtigelsen', 'forskellighed', 'forskning', 'forsørgelse', 'forventninger', 'forår', 'fremgang', 'fremskridt', 'fremtiden', 'fribilletter', 'f

# Opgave 4
**Omskriv** kodesekvensen ovenfor, så I i stedet for et `for loop` laver en sorteret liste over substantiver ved hjælp af `list comprehension`. I kan bruge koden for NER-listerne som inspiration.

**Tip:** rækkefølgen skal være `[output loop1 loop2 condition]`

# Opgave 5

**Lav** tilsvarende lister for **verber** og **adjektiver**.

# 4. WordCloud
Der en mange måder at visualisere tekstdata på. En måderne er at lave en en `word cloud`.

I kan læse mere om mulighederne her: https://github.com/amueller/word_cloud

For at lave en word-cloud skal vi først importere WordCloud-modulet.

Hvis I ikke har modulet installeret kan i køre følgende linje i terminalen:

`conda install -c conda-forge wordcloud`

**Importér** WordCloud-modulet.


In [37]:
from wordcloud import WordCloud # der er flere undermoduler i wordcloud - derfor 'from'-kommandoen

ModuleNotFoundError: No module named 'wordcloud'

WordCloud tager hele strenge som input. Vi skal derfor først have samlet vores liste af substantiver til en samlet `string`. Dette gøres med `.join()`-kommandoen.

In [38]:
sub_string_1 = " ".join(subst_1)

Herefter kan vi lave en word cloud over listen af substantiver. Koden er relativt overskuelig og let at læse.

**Afprøv** sekvensen nedenfor. **Læs** koden og **diskutér**, hvad de enkelte dele betyder.

In [39]:
cloud_sub_1 = WordCloud(width = 1200, height = 800,
                background_color ='white',
                max_words=20,
                min_font_size = 10).generate(sub_string_1)

NameError: name 'WordCloud' is not defined

In [ ]:
plt.imshow(cloud_sub_1, interpolation='bilinear')

# Opgave 6
Hvis I har tid tilovers skal kan I lave substantiv-lister og word-clouds for nogle af de andre taler.